## Load Python modules

In [ ]:
import os
from vina import Vina
from biopandas.pdb import PandasPdb
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import  LinearRegression
import numpy as np
import nglview as nv
import rdkit

## Vina scores without docking (Step-by-step)

### 1. Prepare Ligand File
Formatted ligand files for AutoDock must be in pdbqt format and contain atom types supported by AutoDock, plus extra records that specify rotatable bonds. We can use a package called Meeko to convert a ligand file from PDB or SDF format to PDBQT format. Meeko computes charges for the entire ligand. Hydrogens have to be present in the ligand.  

In [ ]:
## Convert Ligand to PDBQT
!mk_prepare_ligand.py -i ligands/CAT13a.sdf -o ligand.pdbqt

### 2. Prepare Receptor
For receptor preparation, we also need to convert it into pdbqt format using Meeko. AurodDock only keeps the polar hydrogens, unless specifically told otherwise. 

In [ ]:
## Convert Protein to PDBQT
!mk_prepare_receptor.py -i BACE_protein.pdb --write_pdbqt receptor.pdbqt

### 3. Grid space and Docking Center
We have to define a grid space to set the location and extent of the 3D area to be searched during the AutoDock experiment. The search space is defined by specifying a center (docking center), the number of points in each dimension, plus the spacing between points.

The docking center refers to the point where we are supposed to test our docking. Typically, if we can identify a complex containing our target protein and a ligand bound to it, we can locate the center of that ligand and then test our docking with that center for our specific ligand. If a protein-ligand complex structure is not available to calculate the center, choose a center using a visualization tool, like ChimeraX. 

In [ ]:
def get_ligand_center(ligand_file):
    x_list, y_list, z_list = [], [], []
    with open(ligand_file) as f:
        for line in f:
            if line.startswith("ATOM") or line.startswith("HETATM"):
                x = float(line[30:38])
                y = float(line[38:46])
                z = float(line[46:54])
                x_list.append(x)
                y_list.append(y)
                z_list.append(z)
    center = [sum(x_list)/len(x_list), sum(y_list)/len(y_list), sum(z_list)/len(z_list)]
    return center
    
center = get_ligand_center('ligand.pdbqt') ## use ligand.pdbqt file to calculate center

### 4. Calculate Vina energy score for the MD conformations
The protein-ligand complexes we are using in this exercise were collected from large-scale calculations of binding energy using MD simulations with the free energy perturbation (FEP) technique. Let's calculate the Vina score for the conformation generated by the MD simulations. We will also calculate the scores after minimizing the conformations using the AutoDock Vina force field. 

In [ ]:
## Docking Parameters
v = Vina(sf_name='vina') #Name of the scoring function

# Load receptor and ligand
v.set_receptor('receptor.pdbqt')
v.set_ligand_from_file('ligand.pdbqt')

v.compute_vina_maps(center=center, box_size=[16, 16, 16])

In [ ]:
energy = v.score()
print('Score before minimization: %.3f (kcal/mol)' % energy[0])

energy_minimized = v.optimize()
print('Score after minimization : %.3f (kcal/mol)' % energy_minimized[0])

## 5. Combining all the steps
Let's put all the steps together ona single cell, and calculate the scores for another protein-ligand complex. 

In [ ]:
## Variables
REC = "BACE"
LIG = "CAT13b"

## Convert Ligand to PDBQT
!mk_prepare_ligand.py -i ligands/CAT13b.sdf -o ligand.pdbqt

## Convert Protein to PDBQT 
!mk_prepare_receptor.py -i BACE_protein.pdb --write_pdbqt receptor.pdbqt

## Calculate center
center = get_ligand_center('ligand.pdbqt')

## Docking Parameters
v = Vina(sf_name='vina')

# Load receptor and ligand
v.set_receptor('receptor.pdbqt')
v.set_ligand_from_file('ligand.pdbqt')

v.compute_vina_maps(center=center, box_size=[16, 16, 16])

energy = v.score()
energy_minimized = v.optimize()

print(f"{REC},{LIG},{energy[0]:.3f},{energy_minimized[0]:.3f}")

## Write it in a file
result_file = "scores.csv"

## If it doesn't exist, write the header. Header will be necessary to plot them with the experimental values
if not os.path.exists(result_file):
    with open(result_file, "a") as file:
        file.write(f"receptor,ligand,vina,vina_minimized\n")

with open(result_file, "a") as file:
    file.write(f"{REC},{LIG},{energy[0]:.3f},{energy_minimized[0]:.3f}\n")  

In [ ]:
## Print score file
with open(result_file, "r") as file:
    content = file.read()
    print(content)

In [ ]:
## delete result file - Run this cell only if you want to rewrite the scores.csv file from scratch
import os

if os.path.exists(result_file):
    os.remove(result_file)
    print(f"{result_file} deleted successfully.")
else:
    print(f"{result_file} does not exist.")

### 6. Plotting
We have provided 14 ligands for the receptor, and their experimental binding energies (deltaG_scores.csv). If you have calculated a minimized score for all 14 ligands using the previous cell, you can plot them against the experimental binding energies to see the correlation between the score and the experimental values. 

In [ ]:
## Vina scores Vs. deltaG

vina_results = 'scores.csv'
deltaG_results = 'deltaG_scores.csv'

df1 = pd.read_csv(vina_results)
df2 = pd.read_csv(deltaG_results)

df1 = pd.merge(df1, df2, on=["receptor", "ligand"], how='inner')

X = df1['deltaG']
Y = df1[['vina']].values.ravel()
    
# --- Plot ---
plt.figure(figsize=(8, 6))
colors = ['#0072B2', '#E69F00']  # Blue, Orange
markers = ['o', 's']
linestyles = ['--', '-']

# Plot: Vina
plt.scatter(X, Y, color=colors[0], marker=markers[0], alpha=0.7)
reg1 = LinearRegression().fit(X.values.reshape(-1, 1), Y)
x1 = np.linspace(X.min(), X.max(), 100)
y1 = reg1.predict(x1.reshape(-1, 1))
plt.plot(x1, y1, linestyle=linestyles[0], color=colors[0])
plt.plot([], [], linestyle='-', color=colors[0])

# Finalize plot (no square aspect)
plt.xlabel("Experimental ΔG (Kcal/mol)", fontsize=20)
plt.ylabel("Vina Scores (Kcal/mol)", fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(fontsize=18)
plt.grid(True)
plt.tight_layout()

# Save plot
#plot_path = f"plot.png"
#plt.savefig(plot_path, dpi=300)
plt.show()


## 7.Running Docking
Now we are going to run a docking exercise. We will assume that we know the pocket region (docking center) on the receptor, have prepared the PDBQT files for the ligand and the receptor, and we know the size of the grid space (box size). We will save the coordinates of the top nine ligand conformations or poses (n_poses=9) and perform an exhaustive search of the ligand poses in the pocket (exhaustiveness=64). Higher exhaustiveness requires a longer running time; however, it reduces the probability of missing a pose in your search. You can increase the exhaustiveness value while benchmarking your system to see if it produces a ligand with a lower score. exhaustiveness=64 is a good choice for accuracy. 

In [ ]:
## Run docking
v = Vina(sf_name='vina')

# Load receptor and ligand
v.set_receptor('receptor.pdbqt')
v.set_ligand_from_file('ligand.pdbqt')

## Calculate center
center = get_ligand_center('ligand.pdbqt')

v.compute_vina_maps(center=center, box_size=[16, 16, 16]) #Change the box size. Also redefine center, if needed

v.dock(exhaustiveness=64, n_poses=9)
v.write_poses('docked_ligands/docked_ligand.pdbqt', n_poses=9, overwrite=True)

### 8.Splitting the poses
Each output file from Autodock Vina will have the number of poses we requested. We will split the individual poses using the tool openBabel.

In [ ]:
## Convert docked pdbqt to pdb(different poses)
!obabel docked_ligands/docked_ligand.pdbqt -O docked_ligands/docked_ligand.pdb -m
#!obabel docked_ligands/docked_ligand.pdbqt -O docked_ligands/docked_ligand.pdbqt -m

### 9. Visualization
Let's visualize the poses

In [ ]:
## View best pose
view = nv.show_file('docked_ligands/docked_ligand1.pdb')
view.add_component('receptor.pdbqt')
view

In [ ]:
## View all poses
view = nv.NGLWidget()
# Add receptor
view.add_component('receptor.pdbqt')

# Add each ligand pose
for i in range(1, 9):  # Change range based on how many poses you have
    view.add_component(f'docked_ligands/docked_ligand{i}.pdb')

view

### 10. Scores
Let's look at the scores

In [ ]:
## View scores
scores = v.score()  # Returns a list of tuples: (score, rmsd_l.b., rmsd_u.b.)
k=1
for i in scores:
    print(f"Pose {k}: Score = {i} kcal/mol")
    k += 1

#print(v.score())

## 11. Excercise 
You can plot the scores of the best poses with the experimental binding affinity values for all the ligands. Also, you can calculate RMSD values of the best and all the poses with the MD derived structure and plot them against the scores. 

#### Vina Score vs DeltaG
#### Vina Score vs RMSD

## 12. Smina scores
AutoDock Vina combines different interaction terms to produce the score. If we want to examine the individual terms of the interactions (and some additional terms) and create our own scoring function, we can use Smina. Smina requires the pdbqt file of the receptor and the docked ligand, and it returns all the individual scoring terms, which we can use to develop a scoring function specific to our system.

In [ ]:
## Smina
#!smina -r receptor.pdbqt -l ligand.pdbqt --score_only --log score.log #Change the ligand files to get scored of the poses
!smina -r receptor.pdbqt -l docked_ligands/docked_ligand2.pdbqt --score_only >> score.log #Change the ligand files to get scored of the poses
#!smina -r receptor.pdbqt -l docked_ligands/docked_ligand2.pdbqt --score_only --log score.log

In [ ]:
## Print score file
with open("score.log", "r") as file:
    content = file.read()
    print(content)